In [10]:
from wit import Wit
import pandas as pd
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#from bs4 import BeautifulSoup
from collections import defaultdict
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

%matplotlib inline

# 1.Selected_comments_with_witai_sentiment_results
## import comment with side effects for query

In [95]:
SE_tem_l400 = pd.read_csv("wit_ai_sentment/SE_tem_l400.csv",index_col=0,header=None)[1]
SE_tem_l400 = SE_tem_l400.iloc[3:].astype(str)
idxs = list(SE_tem_l400.index)

In [96]:
first_quest2 = pd.read_csv("wit_ai_sentment/SE_tem_l400_first_request.ccv",index_col=0)

In [97]:
print("Total results:",len(first_quest2))
first_quest2.groupby('value').size()

Total results: 30689


value
negative    16765
neutral     12010
positive     1914
dtype: int64

In [57]:
for i_n in first_quest2[first_quest2['value'] == 'negative'].sample(10)['text']:#.iloc[0]
    print(i_n+"\n")

The Belsomra seems to have no side effects for me whatsoever, just a gentle urge to sleep (which can be easily ignored if necessary, unlike a good dose of ambien which is dangerous) and the ability to stay asleep or fall back asleep

I just started this pill a few weeks ago after going off the Nuva ring, which had terrible side effects as well.  I don't know why I get this side effect but I wish I could find a form of birth control that didn't have these side effects for me

I have tried other meds, such as Percocet, oxycodone but those gave me too many side effects

I have had no negative side effects wit this birth control

I have been combatting depression, fatigue, and vaginal dryness since my third shot hoping that I could overcome them or being in denial that is was from the shot, but it's definitely a side effect of the birth control

I've had side effects to them all, but Cipro was absolutely the worst.  These side effects can be permanent, or at least last a long time, maybe e

In [59]:
for i_n in first_quest2[first_quest2['value'] == 'neutral'].sample(10)['text']:#.iloc[0]
    print(i_n+"\n")

I did have some side effects

No side effects

The Remicade treatment gave him few side effects

I also had no side effects

I took Trinessa for a year and had no negative side effects, but I decided to try a few other birth control pills in an attempt to have a shorter period

No side effects

So I kept using it and by the 4th month no mood swings or side effects

Unfortunately, side effects are present, and for me, never went away.  Because of the side effects, I don't particular recommend this medicine, but everyone's body works differently, and it may be right for you

Maybe should take with food to reduce side effects

I am now on Loestrin 24 Fe and have had NONE of these side effects



In [58]:
for i_n in first_quest2[first_quest2['value'] == 'positive'].sample(10)['text']:#.iloc[0]
    print(i_n+"\n")

I haven't noticed any side effects and it has truly been a blessing in my life

I have had no side effects from the drug and my doctor and I have been happy with the results

Works great with no side effects

Works great, no side effects

It has been a godsend for me and I have had zero side effects

I think the side effects are worth it as I finally feel good for the first time in weeks

The first one is I am so happy to have found a birth control that (for me) has very little negative side effects.  All in all these side effects barely bother me in comparison with what I've had to deal with on other birth controls

Oh and I have not had ANY BAD sexual side effects, and my wife is HAPPY

No side effects, all went very well

So far so good, not even any side effects to speak of



# 2. Train the model using only these comment with side effects 

In [20]:
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud

from sklearn.svm import SVC

## only need to remove punctuation and stemize
stemmer = SnowballStemmer('english')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [21]:
## remove special symbol
def rm_sym(df):
    df['review'] = df['review'].str.replace("&#039;",'\'')
    df['review'].head()
    df['rating_cate'] = ''
    df.loc[df['rating'] >= 7,'rating_cate'] = 'high'
    df.loc[df['rating'] <= 4,'rating_cate'] = 'low'
    df.loc[(df['rating'] > 4) & (df['rating'] < 7),'rating_cate'] = 'medium'
    return df

In [22]:
df = pd.read_csv('drugsCom_raw/drugsComTrain_raw.tsv',sep='\t',index_col=0)
df = rm_sym(df)


test = pd.read_csv("drugsCom_raw/drugsComTest_raw.tsv",sep='\t', index_col=0)
test = rm_sym(test)

In [26]:
train = pd.concat([df.loc[first_quest2.index], first_quest2],1)
## Generate table of words with their counts
con_vec = TfidfVectorizer(stop_words='english',tokenizer=tokenize)
X_train_p = con_vec.fit_transform(train['review'])
#y_train = train['rating_cate']


/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [27]:
negnornot = (train['value'] == "negative").astype(int)
negnornot.shape

(30689,)

In [28]:
X_train,X_test,y_train,y_test =  train_test_split(X_train_p,negnornot,test_size=0.25)

## Logistic regression

In [29]:
lr = LogisticRegression(penalty='l1',multi_class='auto',solver='saga')
#lr.fit(X,target_3.iloc[:,0])

cv_score = cross_val_score(lr,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
cv_score

array([0.68886861, 0.67961418, 0.6835245 ])

### RandomForestClassifier

In [30]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=300,n_jobs=-1)
rfc_cv_score = cross_val_score(rfc,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
rfc_cv_score

array([0.7632951 , 0.77111575, 0.7653806 ])

In [31]:
rfc = RandomForestClassifier(n_estimators=300,n_jobs=-1)
rfc.fit(X_train,y_train)
y_test_predict = rfc.predict(X_test)
accuracy_score(y_test,y_test_predict)

0.8118076371692949

In [32]:
feature_importances = pd.DataFrame(rfc.feature_importances_,con_vec.get_feature_names())
feature_importances.sort_values(0,ascending=False)

,0
bad,0.011099
effect,0.009980
headach,0.005513
horribl,0.005404
year,0.005260
work,0.005085
onli,0.004601
day,0.004482
week,0.004296
month,0.004151


### Gradient boosting classifier

In [33]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=300)
gbc_cv_score = cross_val_score(gbc,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

array([0.69721064, 0.68847758, 0.69890511])

### SVM

#### poly

In [34]:
svm_poly = SVC(kernel='poly')
gbc_cv_score = cross_val_score(svm_poly,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


KeyboardInterrupt: 

#### linear

In [ ]:
svm_lin = SVC(kernel='linear')
gbc_cv_score = cross_val_score(svm_lin,X_train,y_train,scoring='accuracy',cv=3,n_jobs=-1)
gbc_cv_score

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
#from bs4 import BeautifulSoup
from collections import defaultdict
#import requests
%matplotlib inline
from sklearn.svm import SVC
from nltk.stem.snowball import SnowballStemmer

## remove special symbol

def clean_text(df_tem3):
    df['review'] = df['review'].str.replace("&#039;",'\'')
    df_tem3['review'] = df_tem3['review'].str.replace("\"","").str.lower()
    df_tem3['review'] = df_tem3['review'].str.replace( r"(\\r)|(\\n)|(\\t)|(\\f)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(&#039;)|(\d\s)|(\d)|(\/)","")
    df_tem3['review'] = df_tem3['review'].str.replace("\"","").str.lower()
    df_tem3['review'] = df_tem3['review'].str.replace( r"(\$)|(\-)|(\\)|(\s{2,})"," ")
    df_tem3['review'].sample(1).iloc[0]

    stemmer = SnowballStemmer('english')
    df_tem3['review'] = df_tem3['review'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split(" ")]))
    return df_tem3


np.random.seed(9)

In [99]:
import tensorflow as tf
import tensorflow 

#from tensorflow import tensorflow.keras

#from keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional



# fix random seed for reproducibility

MAX_NB_WORDS = 500
max_review_length = 500
EMBEDDING_DIM = 160


In [101]:
first_quest3 = first_quest2.copy()
first_quest3.columns = ['review', 'confidence', 'value']
df_tem3 = clean_text(first_quest3)


In [108]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_tem3,test_size=0.2)

In [109]:
# Tokenize the data
tokenizer = Tokenizer(num_words = MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True, split=' ', char_level=False, 
                      oov_token=None, document_count=0)

tokenizer.fit_on_texts(train['review'])
train_sequences = tokenizer.texts_to_sequences(train['review'])
test_sequences = tokenizer.texts_to_sequences(test['review'])

In [110]:
# truncate and pad input sequences
X_train = sequence.pad_sequences(train_sequences, maxlen=max_review_length)
X_test = sequence.pad_sequences(test_sequences, maxlen = max_review_length)

In [112]:
word_index = tokenizer.word_index

print(X_train.shape, '<-- shape of train_data ready for val/train split.')
print(X_test.shape, '<-- shape of final_test_data ready for fedding to network.')
print(len(tokenizer.word_index), '<-- Length of Word Index')

(24551, 500) <-- shape of train_data ready for val/train split.
(6138, 500) <-- shape of final_test_data ready for fedding to network.
6314 <-- Length of Word Index


In [119]:
y_train = (train['value'] == 'negative').astype(int)
y_test = (test['value'] == 'negative').astype(int)
print(y_train.shape, '<-- shape of train_data ready for val/train split.')
print(y_test.shape, '<-- shape of final_test_data ready for fedding to network.')

(24551,) <-- shape of train_data ready for val/train split.
(6138,) <-- shape of final_test_data ready for fedding to network.


In [123]:
nb_words  = min(MAX_NB_WORDS, len(word_index))
lstm_out = max_review_length

model = Sequential()
model.add(Embedding(nb_words,EMBEDDING_DIM,input_length=max_review_length))
#model.add(Dropout(0.2))

## add conv using kernal No.32 and size 3x3, actiation='relu'(rm neg)
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPool1D(pool_size=2))
#model.add(Bidirectional(LSTM(50, return_sequences=True)))
#model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
#model.add(Attention(max_review_length))
model.add(Dense(1, activation = 'sigmoid'))

## one-code mutiple categories targets use 'categorical_crossentropy' not 'binary_crossentropy'
model.compile(loss='binary_crossentropy',optimizer='adam',metrics =['accuracy'])

In [124]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 160)          80000     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 100)               84400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 164,501
Trainable params: 164,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


## if 4 steps without decreasing of loss in valid set, stop the trainning
batch = 32 
epoch = 40

early_stopping = EarlyStopping(patience = 4)

LSTM_model = model.fit(X_train, y_train, batch_size=batch, epochs=epoch,
                       validation_split=0.2,callbacks=[early_stopping], shuffle = True)

best_score = min(LSTM_model.history['val_loss'])

Train on 19640 samples, validate on 4911 samples
Epoch 1/40
19640/19640 [==============================] - 1049s 53ms/sample - loss: 0.4082 - accuracy: 0.8095 - val_loss: 0.3364 - val_accuracy: 0.8564
Epoch 2/40
18400/19640 [===========================>..] - ETA: 1:06 - loss: 0.3182 - accuracy: 0.8666